In [7]:
###-Repositorio estadistica informatica pregrado UNALM-### 

#Trabajo Final → AVANCE.
#1. Relación de repositorios con sus respectivos url
#2. Automatización de la descarga de la información de un repositorio.
#3. Estructuración de la información descargada en formato csv

#repositorios

#1. UNALM Pregrado #http://repositorio.lamolina.edu.pe/handle/20.500.12996/26/recent-submissions
#2. PUCP Posgrado #https://tesis.pucp.edu.pe/repositorio/handle/20.500.12404/757/recent-submissions
#3. UNALM Posgrado #http://repositorio.lamolina.edu.pe/handle/20.500.12996/329


## Repositorio 1 PUCP pregrado 
url = 'https://tesis.pucp.edu.pe/repositorio/handle/20.500.12404/757/recent-submissions'

#paquetes 
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 
import pandas as pd

import ssl
context = ssl._create_unverified_context() #la pagina pide un socket seguro 

#Las tesis se encuentran en 4 paginas distinas 

#devuelve la url de la siguiente pagina 
def next_page(url):
    page = urllib.request.urlopen(url,context = context )
    soup = BeautifulSoup(page, "html.parser")
    
    line = soup.find('a',{'class':'next-page-link'})
    line = line.decode().strip()
    offset = re.findall(r'\?offset=.{2}',line)
    url_diff = offset[0]
    
    url = re.sub('\?offset.*','',url)
    n_url = url + url_diff
    return n_url

URL = []
url1 = url

#devuelve las url de todas las paginas 
def all_pages(url,URL):
    
    page = urllib.request.urlopen(url,context = context)
    soup = BeautifulSoup(page, "html.parser")
    last = soup.find('li',{'class':'next pull-right disabled'})
    URL.append(url)
    if last == None: 
        url = next_page(url)
        all_pages(url,URL)
    else: 
        return URL 
    return URL 
all_pages(url1,URL)

###Almacenar links de las tesis 

handles = [] 

#las url de las tesis se diferencian por el codigo handle/20.500.12404/***** <-- diferencia 
for urls in URL: 
    page = urllib.request.urlopen(urls,context = context)
    soup = BeautifulSoup(page, "html.parser")
    for lines in soup.find_all('h4'):
        lines = lines.decode().strip()
        link = re.findall(r'(?<=20.500.12404/).+?(?=\")',lines) #handle unico a cada tesis (#12996)
        handles.append(link)
       
## Links completos 

#creando las url especificas de cada tesis 
links=[]
def get_links(handles,links):
    handle_url = re.sub('(?<=20.500.12404/).+','',url)
    for i in range(len(handles)): 
        links.append(handle_url+handles[i][0])
    return links

get_links(handles,links)    

#agregando ?show=full a las url se muestra informacion mas detallada de cada tesis 
links_full = [] 
for link in links: 
    links_full.append(link+'?show=full')



In [8]:
## almacenando en diccionario 

rep_pucp = { 
        'unis' : [],
        'titulos' : [],
        'nombres' : [],
        'grados' : [], 
        'asesores' : [],
        'resumenes': [],
        'años' : [] }
 
##buscar informacion especifica en las url de las tesis  
def get_all(links_full,reps_unalm):
    for i in range(len(links_full)): 
        page = urllib.request.urlopen(links_full[i],context = context)
        soup = BeautifulSoup(page)
        full_t=''
        for line in soup.find_all('tr'):
            full_t = line.decode().strip()+full_t
        #se encuentra la informacion 
        uni = re.findall(r'(?<=>thesis\.degree\.grantor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        titulo = re.findall(r'(?<=>dc\.title</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        nombre = re.findall(r'(?<=>dc\.contributor\.author</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        grado = re.findall(r'(?<=>thesis\.degree\.name</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        asesor = re.findall(r'(?<=>dc\.contributor\.advisor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        #algunos resumenes tienen \n asi que se usa [\s\S]
        resumen = re.findall(r'(?<=</tr><tr class=\"ds-table-row even\">\n<td class=\"label-cell\">dc.description.abstract</td><td class=\"word-break\">)[\s\S]*?(?=</td><td>es_ES)',full_t)  
        año = re.findall(r'(?<=>dc\.date\.issued</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        
        rep_unalm['unis'].append(uni[0])
        rep_unalm['titulos'].append(titulo[0])     
        rep_unalm['nombres'].append(nombre[0])
        rep_unalm['grados'].append(grado[0]) 
        
        #estas dos variables generaron N/A en algun momento del codigo 
        if asesor == []: 
            rep_unalm['asesores'].append('Sin asesor')
        else:
            rep_unalm['asesores'].append(asesor[0])
        if resumen == []: 
            rep_unalm['resumenes'].append('Sin resumen')
        else:
            rep_unalm['resumenes'].append(resumen[0])
            
        rep_unalm['años'].append(año[0])
    return reps_unalm

universities = get_all(links_full,rep_unalm)


In [26]:
df = pd.DataFrame(universities)
df.to_csv('tesisPUCPfinal.csv', index=False,encoding='utf-8-sig',
          header = ['Institución','Título de la tesis', 'Nombre del tesista', 'Grado', 'Nombre del asesor', 'Resumen','Año'])

df_read = pd.read_csv('tesisPUCPfinal.csv')

for resumen in df_read.loc[3:5,'Resumen']:
    print(resumen) 


La presente tesis propone un modelo de regresi on cuant  lica en d onde la variable es no
negativa y posee censura intervalar, es decir que esta no es directamente observable, y la  unica
informaci on que se conoce sobre ella es que se encuentra en cierto intervalo. Para evaluar si
la metodolog  a de estimaci on captura adecuadamente los par ametros poblacionales desde el
punto de vista de la inferencia cl asica, se desarrolla un estudio de simulaci on. Finalmente, se
aplica el modelo a los datos de la Encuesta Nacional de Satisfacci on de Salud ejecutada el
a~no 2015. La estructura del modelo permite evaluar los factores relacionados al sueldo de los
profesionales en salud (el cual hab  a sido censurado desde el proceso de recolecci on de datos).
El presente modelo es una extensi on al modelo de regresi on de censura intervalar expuesto
en Sal y Rosas et al. (2019), pues eval ua los factores subyacentes a una variable respuesta a
lo largo de sus cuantiles.
En la tariﬁcación de seguros